In [11]:
import mne
import numpy as np
import xarray as xr

from megspikes.pipeline import (iz_prediction_pipeline,
                                read_detection_iz_prediction_pipeline)
from .utils import setup_case_manager
from .convert_from_nix_format import convert_clusters_from_nix_format
from megspikes.database.database import (check_and_read_from_dataset,
                                         check_and_write_to_dataset)
from megspikes.localization.localization import PredictIZClusters
from megspikes.visualization.visualization import ClusterSlopeViewer
from megspikes.utils import prepare_data

In [ ]:
subj = 6

case = setup_case_manager(subj)
raw = mne.io.read_raw_fif(case.fif_file)

In [21]:
path = case.cluster_dataset.with_name(f'{case.case}_clusters_manually_checked.nc')
with xr.open_dataset(path) as ds:
    clusters_checked = ds.load()

In [22]:
(clusters_checked.iz_prediction.loc[:, 'peak'].values > 0).sum()

465

In [ ]:
case.manual_cluster_dataset

Convert cluster results

In [ ]:
for subj in [6]:
    case = setup_case_manager(subj)
    raw = mne.io.read_raw_fif(case.fif_file, preload=True)
    raw_filt = prepare_data(raw.copy(), meg=True, filtering=[2, 90, 50], resample=None, alpha_notch=None)
    clusters, pc = convert_clusters_from_nix_format(case, raw_filt)

In [14]:
predict = PredictIZClusters(
            case=case,
            sensors=True,
            smoothing_steps_one_cluster=3,
            smoothing_steps_final=10,
            amplitude_threshold=0.5,
            min_sources=10)
clusters, _ = predict.fit_transform((clusters, None))
(clusters.iz_prediction.loc[:, 'peak'].values > 0).sum()

839

In [16]:
(clusters.iz_prediction.loc[:, 'slope'].values > 0).sum()

134

In [ ]:
%matplotlib qt5


path = case.cluster_dataset.with_name(f'{case.case}_clusters_results_converted.nc')
with xr.open_dataset(path) as ds:
    clusters_checked = ds.load()
    
pc = ClusterSlopeViewer(clusters_checked, case)
app = pc.view()
app

In [ ]:
(clusters_checked.iz_prediction.loc[:, 'peak'].values > 0).sum()